# 03 - Detección en video


![opencv-tensor](../images/opencv-tensor.webp)

In [ ]:
#librerias

import cv2

from tensorflow.keras.models import load_model

from imutils.video import VideoStream
import imutils

import time

from src.detector import detectar

In [ ]:
# tematica a detectar

tema='mascara'

In [ ]:
# se carga el modelo detector de caras

prototxt='model/deploy.prototxt'

pesos='model/res10_300x300_ssd_iter_140000.caffemodel'

net_cara=cv2.dnn.readNet(prototxt, pesos)

In [ ]:
# se carga el modelo detector del tema

net_tema=load_model(f'model/mascara.h5')

In [ ]:
# inicializa camara web

def camara():

    cam=VideoStream(src=0).start()
    time.sleep(2)

    # bucle sobre los frames
    while 1:
        # coge el frame del video y redimensiona a 400 pixels
        frame=cam.read()
        frame=imutils.resize(frame, width=400)

        # detectar caras en el frame y determinar si hay o no tema
        locs, preds=detectar(frame, net_cara, net_tema)

        # bucle sobre las caras detectadas y sus localizaciones
        for caja, pred in zip(locs, preds):
            x_start, y_start, x_end, y_end=caja
            con, sin=pred

            # determinar la etiqueta y color para dibujar caja y texto (verde y rojo)
            etiqueta='mascara' if con > sin else 'Sin mascara'
            color=(0, 255, 0) if etiqueta==tema else (0, 0, 255)  # B-G-R

            # incluye la probabilidad en la etiqueta
            etiqueta='{}: {:.2f}%'.format(etiqueta, max(con, sin)*100)

            # enseña la etiqueta y el limite de la caja en el frame
            cv2.putText(frame, 
                        etiqueta, 
                        (x_start, y_start-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        .45, color, 2)
            
            cv2.rectangle(frame, (x_start, y_start), (x_end, y_end), color, 2)

        # muestra el frame de salida
        cv2.imshow('Camara', frame)
        key=cv2.waitKey(1)

        # si se presiona la tecla q, rompe el bucle
        if key==ord('q'):
            break

    # limpia pantalla
    cv2.destroyAllWindows()
    cam.stop()


In [ ]:
camara()